In [1]:
# Importing neccesary libraries for the project
import pandas as pd
import datetime 

In [2]:
# Reading the csv file & storing it as a Dataframe object with the variable name "online"
online = pd.read_csv('online_data.csv')

#Conversion of dtype of InvoiceDate column from 'object' to 'datetime' type for better date/time manipulations:
online.InvoiceDate =  pd.to_datetime(online.InvoiceDate)

In [4]:
online.head()

,InvoiceNo,StockCode,InvoiceDate,CustomerID,Quantity,UnitPrice
0,572558,22745,2011-10-25 08:26:00,14286,2,1740
1,577485,23196,2011-11-20 11:56:00,16360,2,3177
2,560034,23299,2011-07-14 13:35:00,13933,2,3429
3,578307,72349B,2011-11-23 15:53:00,17290,1,2298
4,554656,21756,2011-05-25 13:36:00,17663,3,1388


# Data preparation steps

In [5]:
#We're starting with a pre-processed online DataFrame with only the latest 12 months of data:

print('Min:{}; Max:{}'.format(min(online.InvoiceDate),max(online.InvoiceDate)))


Min:2010-12-01 08:26:00; Max:2011-12-09 12:49:00


In [6]:
# creating a hypothetical snapshot_day data as if we're doing analysis recently.

snapshot_date = max(online.InvoiceDate) + datetime.timedelta(days=1)


In [7]:
type(snapshot_date)

pandas._libs.tslibs.timestamps.Timestamp

## Calculating Total Sum amount in each transaction

In [8]:
online['TotalSum']= online['Quantity']*online['UnitPrice']

online.head()

,InvoiceNo,StockCode,InvoiceDate,CustomerID,Quantity,UnitPrice,TotalSum
0,572558,22745,2011-10-25 08:26:00,14286,2,1740,3480
1,577485,23196,2011-11-20 11:56:00,16360,2,3177,6354
2,560034,23299,2011-07-14 13:35:00,13933,2,3429,6858
3,578307,72349B,2011-11-23 15:53:00,17290,1,2298,2298
4,554656,21756,2011-05-25 13:36:00,17663,3,1388,4164


# Calculating the metrics : Recency, Frequency & Monetary Value

In [9]:
# Aggregating data on a customer level
datamart = online.groupby(['CustomerID']).agg({'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
                                               'InvoiceNo': 'count',
                                               'TotalSum': 'sum'})

# Renaming columns for easier interpretation
datamart.rename(columns = {'InvoiceDate': 'Recency',
                           'InvoiceNo': 'Frequency',
                           'TotalSum': 'MonetaryValue'}, inplace=True)

# Checking the first 5 rows:
datamart.head()

,Recency,Frequency,MonetaryValue
CustomerID,,,
12747,2,27,121712
12748,1,967,5625961
12749,4,37,277003
12820,3,17,114433
12822,71,9,59892


# Calculating Recency quartile & assigning it as a new column in the datamart

In [11]:
r_labels = range(4, 0, -1)
r_quartiles = pd.qcut(datamart['Recency'], 4, labels = r_labels)
datamart = datamart.assign(R = r_quartiles.values)

In [12]:
type(r_quartiles)

pandas.core.series.Series

# Calculating Frequency and Monetary value quartiles in similar way

In [18]:
f_labels = range(1,5)
m_labels = range(1,5)
f_quartiles = pd.qcut(datamart['Frequency'], 4, labels = f_labels)
m_quartiles = pd.qcut(datamart['MonetaryValue'], 4, labels = m_labels)
datamart = datamart.assign(F = f_quartiles.values)
datamart = datamart.assign(M = m_quartiles.values)

In [19]:
type(f_quartiles)

pandas.core.series.Series

In [20]:
datamart.F.dtypes

CategoricalDtype(categories=Noneordered=True)

# Build RFM Segment and RFM Score

In [23]:
# Concatenating R F M quartile values to RFM_Segment
# Summing RFM quartiles values to RFM_Score

def join_rfm(x): 
    return str(x['R']) + str(x['F']) + str(x['M'])


datamart['RFM_Segment'] = datamart.apply(join_rfm, axis=1)

datamart['RFM_Score'] = datamart[['R','F','M']].sum(axis=1)

datamart.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12747,2,27,121712,4,4,3,443,11.0
12748,1,967,5625961,4,4,4,444,12.0
12749,4,37,277003,4,4,4,444,12.0
12820,3,17,114433,4,3,3,433,10.0
12822,71,9,59892,2,2,3,223,7.0


# Largest RFM segments

In [24]:
datamart.groupby('RFM_Segment').size().sort_values(ascending = False)[:10]

RFM_Segment
444    462
111    436
344    240
211    234
333    222
233    222
433    219
222    206
122    199
322    154
dtype: int64

# Filtering on RFM segments

In [28]:
# Selecting bottom RFM segment "111" and viewing top 5 rows:

datamart[datamart['RFM_Segment']=='111'][:5]

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12823,297,1,10719,1,1,1,111,3.0
12829,337,2,6548,1,1,1,111,3.0
12837,173,2,9615,1,1,1,111,3.0
12852,295,2,12198,1,1,1,111,3.0
12873,282,1,9936,1,1,1,111,3.0


# Summary metrics per RFM Score

In [29]:
datamart.groupby('RFM_Score').agg({'Recency': 'mean',
                                   'Frequency': 'mean',
                                   'MonetaryValue': ['mean', 'count'] }).round(1)

Recency Frequency MonetaryValue      
             mean      mean          mean count
RFM_Score                                      
3.0         257.4       2.0       10814.5   436
4.0         133.4       2.6       15540.4   329
5.0         148.0       4.5       26310.3   411
6.0          80.4       5.5       32627.9   364
7.0         102.0       9.0       53395.0   351
8.0          66.9      11.9       71354.7   373
9.0          53.8      16.2       97600.5   326
10.0         34.2      22.2      132158.3   372
11.0         29.1      41.2      247846.1   277
12.0          8.0      70.8      425467.8   462

# Grouping into named segments

In [30]:
# Using RFM score for grouping customers into Gold, Silver and Bronze segments:

def segment_me(df):
    if df['RFM_Score'] >= 9:
        return 'Gold'
    elif (df['RFM_Score'] >= 5) and (df['RFM_Score'] < 9):
        return 'Silver'
    else:
        return 'Bronze'

In [31]:
datamart['General_Segment'] = datamart.apply(segment_me, axis=1)

In [32]:
datamart.groupby('General_Segment').agg({'Recency': 'mean',
                                         'Frequency': 'mean',
                                         'MonetaryValue': ['mean', 'count']}).round(1)

Recency Frequency MonetaryValue      
                   mean      mean          mean count
General_Segment                                      
Bronze            204.1       2.3       12846.9   765
Gold               29.2      40.1      240918.7  1437
Silver            100.6       7.6       45394.9  1499